In [2]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate
!pip install stable_diffusion_videos

In [3]:
import os 
import pandas as pd
import numpy as np

from google.colab import auth

import gspread
from google.auth import default

import torch
from diffusers import StableDiffusionPipeline


In [4]:
connect_google_drive = True #@param {type:"boolean"}

if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)


Mounted at /content/gdrive


In [5]:

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )  


pipe = pipe.to("cuda")

from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [6]:

code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'

prompt_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1YhC-iejPPC0bV4OJlxqZIQJj3FJqM-6xYauY7TB8EQA')

sheets = prompt_sheet.worksheet('prompts').get_all_values()
df_prompt = pd.DataFrame(sheets[1:], columns=sheets[0]).set_index('name')


In [7]:
df_prompt

,prompt,seeds,guidance_scale
name,,,
succulent,"a giant succulent in the desert in the sand, t...",5842234013956479,5
reflected galaxy,underground lake with a galaxy reflected in th...,5883559517894420,50
red cave1,"descending deep into a cave, red lava on the w...",4866412378884770 4940561922459094,7.5
alien dunes,rolling dunes on an alien planet at night vivi...,5453396741977211 6583682090290432 648251554957...,7.5
cornu,"standing wave pattern, rainbow, wishing well",5842234013956479,50
alien mountain,"inspired by David A. Hardy, digital landscape ...",4092123727786949 7285707432238947,7.5
overgrown fantasy,a foggy view of a bridge in a fantasy overgrow...,5618257995059877 4638125635335835 823596687782...,7.5
falling in cave,"descending deep into a cave, red lava on the w...",5451964648290255,7.5
vine cave,a foggy view in a fantasy overgrown vines worl...,7078380737840840,7.5


In [21]:
prompt_name = 'fractal vines'
prompt = df_prompt['prompt'][prompt_name]
guidance_scale = float(df_prompt['guidance_scale'][prompt_name])
seed = 2922127385296522  

width = 512
height = 512

latent_width = width // 8
latent_height = height // 8

generator = torch.Generator(device="cuda")

generator.manual_seed(seed)

latent_choose = torch.randn(
    (1, pipe.unet.in_channels, latent_height, latent_width),
    generator = generator,
    device = "cuda"
)


In [22]:
def make_latent_steps(start_latent, stop_latent, steps):
  delta_latent = (stop_latent - start_latent)/steps
  return [start_latent + delta_latent*i for i in range(steps)]

In [23]:
from IPython.display import clear_output


code_folder = r'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes'
output_dir = os.path.join(code_folder, 'output_latent_noise', "{}_{}".format(prompt_name, seed))
if not os.path.exists(output_dir): os.makedirs(output_dir)


num_intermediate_steps = 10


scales = [0.2,0.2,0.2,0.2,0.2,0.25,0.3,0.5,0.5,0.6,1]
num_images = len(scales)

duration = 30
fps = (num_intermediate_steps*num_images)/duration

# num_images = 5
# scale = 0.25 # Scale of noise

j=0 

current_latent = latent_choose

for i in range(num_images):

  latent_noise = torch.randn(
    (1, pipe.unet.in_channels, latent_height, latent_width),
    device = "cuda"
  )

  scale = scales[i]
  target_latent = latent_choose + latent_noise*scale

  for latent_step in make_latent_steps(current_latent, target_latent, num_intermediate_steps):

    j += 1

    clear_output(wait=True)
    print("Making image {} out of {} within batch {}".format(j, num_images*num_intermediate_steps, i))

    with torch.autocast("cuda"):
        images = pipe(
            [prompt],
            guidance_scale=guidance_scale,
            latents = latent_step,
        )

    output_image = images.images[0]

    # for i, image in enumerate(images.images):
    output_image.save(os.path.join(output_dir, '{0:04d}.png'.format(j)))

  current_latent = target_latent



from stable_diffusion_videos import make_video_pyav



Making image 110 out of 110 within batch 10


  0%|          | 0/50 [00:00<?, ?it/s]

In [24]:
make_video_pyav(output_dir, 
                output_filepath=os.path.join(code_folder, 'output_latent_noise', "{}.mp4".format(seed)),
                fps=fps
                )

'/content/gdrive/MyDrive/AI Music Visuals Share/New Codes/output_latent_noise/2922127385296522.mp4'